In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("../data/raw/data.csv")
display(df.head())

df_dict = pd.read_csv("../data/external/dictionary.csv")
display(df_dict)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


,variavel,descricao,tipo,subtipo
0,species,Nome da espécie,qualitativa,nominal
1,island,Nome da ilha onde foi feita a medição,qualitativa,nominal
2,bill_length_mm,Comprimento de bico,quantitativa,continua
3,bill_depth_mm,Profundidade (altura) de bico,quantitativa,continua
4,flipper_length_mm,Comprimento de nadadeira,quantitativa,continua
5,body_mass_g,Massa corporal (peso),quantitativa,continua
6,sex,Sexo dos indivíduos,qualitativa,nominal


In [3]:
target_variable = 'species'
quantitative_variables = (
    df_dict
    .query("tipo == 'quantitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)
qualitative_variables = (
    df_dict
    .query("tipo == 'qualitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [4]:
X = df.drop(columns=[target_variable])
y = df[[target_variable]]

In [5]:
# tratamento de dados discrepantes
quantitative_preprocess = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # tratamento de dados ausentes
    ('normalization', StandardScaler()), # normalização
])

qualitative_preprocess = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados ausentes
    ('encoding', OneHotEncoder()), # transformação de variáveis
    # ('normalization', StandardScaler()), # normalização
])

preprocess = ColumnTransformer([
    ('quantitative', quantitative_preprocess, quantitative_variables),
    ('qualitative', qualitative_preprocess, qualitative_variables)
])